# 1. Linear Regression

solve problem:

$y^{(i)} = \theta^T x^{(i)} + \epsilon^{(i)}$

# 1.1 Assumptions
- errors ~ $N(0, \delta^2)$, that is, $p(\epsilon^{(i)}) = \frac{1}{\sqrt {2\pi} \delta} exp(-\frac{(\epsilon ^{(i)})^2}{2\delta^2})$
    - with rearangement, we have $p(y^{(i)}|x^{(i); \theta}) = \frac{1}{\sqrt {2\pi} \delta} exp(-\frac{(y^{(i)} - \theta^Tx^{(i)})^2}{2\delta^2})$
- samples: independent

## 1.2 Likelihood Function

**Likelihodd Function**:

\begin{equation}

    L(\theta) = p(y^{(1,...,m)} | x^{(1,...,m)}; \theta) = \Pi_{i=1}^m p(y^i|x^i, \theta)
    
\end{equation}

**Notes**
- what parameter can predict the given target when combined with given features -> purpose of maximum likelihood
- the maximum probability that the parameter $\theta$ can make ALL the prediction matches ALL the measurement
- due to the independency of samples, the above equation holds.


**Log-likelihood Function**

\begin{align}
 log L(\theta) &= \sum_{i=1}^{m} log (p(y^i|x^i; \theta)) \\
    &= \sum_{i=1}^{m} log (\frac{1}{\sqrt {2\pi} \delta} exp(-\frac{(y^i - \theta^T(x^i)^2}{2\delta^2})) \\
    &= m log (\frac{1}{\sqrt{2\pi}\delta}) - \frac{1}{\delta^2} \cdot \frac{1}{2} \sum_{i=0}^{m} (y^i - \theta^Tx^i)^2 \\
    &= constant - \frac{1}{\delta^2} \cdot J(\theta)
\end{align}

where $constant$ is a constant value, and $J(\theta)$ is typicall known as the least-sqaures cost function.

**Notes**
- easy calculation by changing production to additions.
